## Instalando dependencias necesarias

In [1]:
%%capture
%pip install pandas
%pip install -U sentence-transformers pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

c:\Users\dido0\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## Entendiendo el dataset

In [2]:
# https://www.kaggle.com/datasets/omarhanyy/imdb-top-1000?resource=download
# TODO: Cargar el archivo del dataset con  pandas

In [2]:
df = pd.read_csv('./resources/IMDB top 1000.csv')
print(df.head())
# TODO: mostrar los primeros 5 registros de dataframe

   Unnamed: 0                                              Title Certificate  \
0           0                 1. The Shawshank Redemption (1994)           R   
1           1                            2. The Godfather (1972)           R   
2           2                          3. The Dark Knight (2008)       PG-13   
3           3                   4. The Godfather: Part II (1974)           R   
4           4  5. The Lord of the Rings: The Return of the Ki...       PG-13   

  Duration                     Genre  Rate  Metascore  \
0  142 min                     Drama   9.3       80.0   
1  175 min              Crime, Drama   9.2      100.0   
2  152 min      Action, Crime, Drama   9.0       84.0   
3  202 min              Crime, Drama   9.0       90.0   
4  201 min  Action, Adventure, Drama   8.9       94.0   

                                         Description  \
0  Two imprisoned men bond over a number of years...   
1  The aging patriarch of an organized crime dyna...   
2  When 

## Usando Sentence Transformer para crear embeddings

In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [6]:
embeddings = model.encode(df['Description'],batch_size=64,show_progress_bar=True)
#embeddings = model.encode(df['Title'],batch_size=64,show_progress_bar=True)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches: 100%|██████████| 16/16 [00:02<00:00,  7.09it/s]


In [10]:
df['embeddings'] = embeddings.tolist()
print(embeddings)

[[-0.06326339  0.04146248 -0.04707528 ... -0.03055874 -0.01641737
  -0.01581548]
 [-0.02941306  0.01056768 -0.06267845 ... -0.01904772  0.03727178
  -0.09891069]
 [ 0.0188879   0.05393049 -0.07068748 ...  0.07443296  0.05758373
  -0.01486044]
 ...
 [-0.01988258  0.015605   -0.02492197 ...  0.00143385 -0.00423387
  -0.00137197]
 [-0.09367283  0.00152869 -0.00855557 ... -0.05445856  0.00994894
   0.03317662]
 [-0.06126129 -0.03122352  0.01081085 ... -0.0570876   0.02968132
  -0.03955578]]


## Calculando la similitud usando la métrica de similitud por coseno

In [8]:
def compute_similarity(example, query_embedding):
    embedding = example['embeddings'] 
    similarity = util.cos_sim(embedding, query_embedding).item()
    return similarity

## Ejecutando la búsqueda 

In [28]:
query_embedding = model.encode(['a travel time adventure'])[0]
df['similarity'] = df.apply(lambda x: compute_similarity(x, query_embedding), axis=1)

# Aumentar contexto --> "Certificate" + "Genre"
df['Certificate | Genre'] = df['Certificate'].astype(str) + ' | ' + df['Genre'].astype(str)


df = df.sort_values(by='similarity', ascending=False)

df.head()[['Title','Certificate | Genre']]
#df.head()[['Title','Classification']].drop_duplicates()

,Title,Certificate | Genre
585,388. Central Station (1998),R | Drama
745,348. Darbareye Elly (2009),"TV-PG | Drama, Mystery, Thriller"
134,135. Klaus (2019),"PG | Animation, Adventure, Comedy"
303,306. Ace in the Hole (1951),"Approved | Drama, Film-Noir"
126,127. Metropolis (1927),"Not Rated | Drama, Sci-Fi"


In [11]:
df[df['Title'] == '73. The Shining (1980)']

,Unnamed: 0,Title,Certificate,Duration,Genre,Rate,Metascore,Description,Cast,Info,embeddings,similarity
72,72,73. The Shining (1980),R,146 min,"Drama, Horror",8.4,66.0,A family heads to an isolated hotel for the wi...,Director: Stanley Kubrick | Stars: Jack Nichol...,"Votes: 877,721 | Gross: $44.02M","[-0.02941306307911873, 0.010567683726549149, -...",0.417388


In [12]:
df.loc[72,'Description']

'A family heads to an isolated hotel for the winter where a sinister presence influences the father into violence, while his psychic son sees horrific forebodings from both past and future.'

In [16]:
df[df['Title'] == '97. Snatch (2000)']
df[df['Title'] == '231. No Country for Old Men (2007)']

,Unnamed: 0,Title,Certificate,Duration,Genre,Rate,Metascore,Description,Cast,Info,embeddings,similarity
228,228,231. No Country for Old Men (2007),R,122 min,"Crime, Drama, Thriller",8.1,91.0,Violence and mayhem ensue after a hunter stumb...,"Directors: Ethan Coen, Joel Coen | Stars: Tomm...","Votes: 838,551 | Gross: $74.28M","[-0.0059683131985366344, 0.02555251121520996, ...",0.212749
